# Hierarchical Dispatch

This notebook illustrates an example of how to use Osier's `LogicDispatchModel`.

In [51]:
# basic imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from unyt import MW, GW, km, unyt_array

# osier imports
from osier import LogicDispatchModel
from osier.tech_library import nuclear_adv, wind, battery, natural_gas
from osier import total_cost, annual_emission

# pymoo imports
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.optimize import minimize
from pymoo.visualization.pcp import PCP

In [2]:
N=24
demand = np.ones(N)*100
wind_speed = np.random.weibull(a=2.5,size=N)

In [10]:
nuclear_adv.capacity = 50
wind.capacity = 20
battery.capacity = 10
natural_gas.capacity = 50

In [11]:
tech_list = [nuclear_adv, wind, battery, natural_gas]
tech_list

[Nuclear_Adv: 50.0 MW,
 WindTurbine: 20.0 MW,
 Battery: 10.0 MW,
 NaturalGas_Conv: 50.0 MW]

In [13]:
net_demand = demand - wind_speed*wind.capacity.to_value()
net_demand

array([71.02468082, 73.61681993, 79.4082994 , 79.89648718, 86.85434205,
       71.2546886 , 83.20470555, 80.58183091, 85.93987225, 78.6084282 ,
       93.58884001, 89.97595819, 82.90419282, 79.7322697 , 79.07432187,
       82.48245396, 64.52157604, 80.85278169, 82.5195001 , 91.5981539 ,
       87.24880579, 75.27437694, 88.66551433, 86.2601746 ])

In [20]:
model = LogicDispatchModel(technology_list=[t for t in tech_list if t.dispatchable],
                           net_demand=net_demand)

In [21]:
model.solve()

unyt_array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0.], 'MW')

In [22]:
model.results

In [27]:
for t in model.technology_list:
    print(t.power_history)

[unyt_quantity(50., 'MW'), unyt_quantity(50., 'MW'), unyt_quantity(50., 'MW'), unyt_quantity(50., 'MW'), unyt_quantity(50., 'MW'), unyt_quantity(50., 'MW'), unyt_quantity(50., 'MW'), unyt_quantity(50., 'MW'), unyt_quantity(50., 'MW'), unyt_quantity(50., 'MW'), unyt_quantity(50., 'MW'), unyt_quantity(50., 'MW'), unyt_quantity(50., 'MW'), unyt_quantity(50., 'MW'), unyt_quantity(50., 'MW'), unyt_quantity(50., 'MW'), unyt_quantity(50., 'MW'), unyt_quantity(50., 'MW'), unyt_quantity(50., 'MW'), unyt_quantity(50., 'MW'), unyt_quantity(50., 'MW'), unyt_quantity(50., 'MW'), unyt_quantity(50., 'MW'), unyt_quantity(50., 'MW')]
[]
[unyt_quantity(21.02468082, 'MW'), unyt_quantity(23.61681993, 'MW'), unyt_quantity(29.4082994, 'MW'), unyt_quantity(29.89648718, 'MW'), unyt_quantity(36.85434205, 'MW'), unyt_quantity(21.2546886, 'MW'), unyt_quantity(33.20470555, 'MW'), unyt_quantity(30.58183091, 'MW'), unyt_quantity(35.93987225, 'MW'), unyt_quantity(28.6084282, 'MW'), unyt_quantity(43.58884001, 'MW'), 

In [28]:
print(model.technology_list)

[Nuclear_Adv: 50.0 MW, Battery: 10.0 MW, NaturalGas_Conv: 50.0 MW]


In [36]:
model.technology_list[0].reset_history()

In [43]:
np.ndarray(model.technology_list[2].power_history)

TypeError: only integer scalar arrays can be converted to a scalar index

In [48]:
len(unyt_array(model.technology_list[0].power_history).to_ndarray())

0

In [47]:
model.net_demand

unyt_array([71.02468082, 73.61681993, 79.4082994 , 79.89648718, 86.85434205,
       71.2546886 , 83.20470555, 80.58183091, 85.93987225, 78.6084282 ,
       93.58884001, 89.97595819, 82.90419282, 79.7322697 , 79.07432187,
       82.48245396, 64.52157604, 80.85278169, 82.5195001 , 91.5981539 ,
       87.24880579, 75.27437694, 88.66551433, 86.2601746 ], 'MW')

In [56]:
battery.technology_type == 'storage'

True

In [57]:
battery.storage_history

[]

In [49]:
data = {}
for t in model.technology_list:
    if t.technology_type == 'storage':
        
    if len(t.power_history) == 0:
        data[t.technology_name] = np.zeros(len(model.net_demand))
    else:
        data[t.technology_name] = unyt_array(t.power_history).to_ndarray()
results = pd.DataFrame()

In [52]:
pd.DataFrame(data)

,Nuclear_Adv,Battery,NaturalGas_Conv
0,0.0,0.0,21.024681
1,0.0,0.0,23.616820
2,0.0,0.0,29.408299
3,0.0,0.0,29.896487
4,0.0,0.0,36.854342
5,0.0,0.0,21.254689
6,0.0,0.0,33.204706
7,0.0,0.0,30.581831
8,0.0,0.0,35.939872
9,0.0,0.0,28.608428


In [31]:
battery.technology_name

'Battery'